## Imports

In [16]:
import pandas as pd

In [24]:
data = pd.read_csv('../data/processed_trips_2018.csv')

In [25]:
data.head()

,tripduration,starttime,stoptime,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,pickup_hour,pickup_day,dropoff_hour,dropoff_day
0,196,2018-01-01 00:01:50.650,2018-01-01 00:05:07.438,315.0,40.703554,-74.006702,259.0,40.701221,-74.012342,18534,Subscriber,1997,1,0,2018-01-01,0,2018-01-01
1,207,2018-01-01 00:02:43.918,2018-01-01 00:06:11.383,3224.0,40.739974,-74.005139,470.0,40.743453,-74.000040,19651,Subscriber,1978,1,0,2018-01-01,0,2018-01-01
2,613,2018-01-01 00:03:15.116,2018-01-01 00:13:28.480,386.0,40.714948,-74.002345,2008.0,40.705693,-74.016777,21678,Subscriber,1982,1,0,2018-01-01,0,2018-01-01
3,375,2018-01-01 00:06:43.709,2018-01-01 00:12:59.450,466.0,40.743954,-73.991449,325.0,40.736245,-73.984738,29822,Subscriber,1982,1,0,2018-01-01,0,2018-01-01
4,402,2018-01-01 00:06:56.924,2018-01-01 00:13:39.792,438.0,40.727791,-73.985649,380.0,40.734011,-74.002939,30722,Subscriber,1989,1,0,2018-01-01,0,2018-01-01


## Clustering

In [26]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20)
kmeans.fit(data[['start_station_latitude', 'start_station_longitude']])
data['start_station_cluster'] = kmeans.labels_

In [27]:
data['start_station_cluster'].value_counts()

17    1752340
13    1675771
1     1643929
12    1397442
7     1333887
18    1308924
4     1166730
3     1027523
15     853106
14     851444
8      809918
0      762644
11     627247
2      595312
19     506873
16     486945
9      263765
6      248723
10     235735
5          81
Name: start_station_cluster, dtype: int64

In [28]:
clusters = data.groupby(['start_station_cluster', 'pickup_day', 'pickup_hour']).size().reset_index(name='trip_count')